# Initial

In [1]:
# Random in position task in list task

# Random in task assign resource (workforce and equipment) to task

# Contraints

In [2]:
# Workforce not performance same time unit at more one task
def avaibale_workforce_contraints():
    # workforce must avaiable at timepoint (available in workloads, available in workschedule)
    pass


# Required skills on task must exited on workforce's skills and workforce's level skills must >= Required level skills
def avaibale_skills_contraints(required_skills, workforce_skills):
    pass

def avaibale_equipments_contraints(required_equipments, equipment_inventory):
    # Required equipments on task must equal equipments assigned

    # Equipments must avaiable at time unit on task
    pass


# Complete time must <= Deadline
def deadline_contraints(end_time, deadline):
    pass

# Objectives

# NSGA2

In [4]:
import pandas as pd
import numpy as np
import time
import pandas as pd
import copy

In [55]:
import pandas as pd
import numpy as np

# Set the number of rows and columns
num_task = 30
num_vm = 10

#-----Genetic Algorithm Paramaters ----------------
population_size= 100
num_iteration=500
crossover_rate= 0.8
mutation_rate= 0.3
mutation_selection_rate= 0.4
num_mutation_jobs=round(num_task*mutation_selection_rate)



# Create column labels (VM1 to VM40)
columns = ['VM{}'.format(i) for i in range(1, num_vm+1)]

# Create row labels (T1 to T100)
rows = ['T{}'.format(i) for i in range(1, num_task+1)]

# Create a random DataFrame
data = np.random.rand(num_task, num_vm)

# Create the Pandas DataFrame
Tasks_Vms_Cost = pd.DataFrame(data, index=rows, columns=columns)

In [49]:
num_mutation_jobs

12

In [57]:
# Create a random DataFrame
data = np.random.uniform(0.95, 1, size=(num_task, num_vm))
Tasks_Vms_Reliability  = pd.DataFrame(data, index=rows, columns=columns)

In [59]:
Task_Vm_Cost=[list(map(float, Tasks_Vms_Cost.iloc[i])) for i in range(num_task)]
Task_Vm_Reliability=[list(map(float,Tasks_Vms_Reliability.iloc[i])) for i in range(num_task)]
# start_time = time.time()

In [61]:
len(Task_Vm_Reliability)

30

## Non-dominated sorting

In [ ]:
def dominate_strategy():

    pass

def non_dominated_sorting(population_size,chroms_obj_record):
    
    s, n = {},{} # s is stored solutions, n is count solutions dominate p
    
    front,rank={},{} 
    
    front[0]=[]     
    
    for p in range(population_size*2):
        s[p]=[]
        n[p]=0
        
        for q in range(population_size*2):
            
            # if q not dominate p
            if ((chroms_obj_record[p][0]>chroms_obj_record[q][0] and chroms_obj_record[p][1]<chroms_obj_record[q][1]) 
            or (chroms_obj_record[p][0]>=chroms_obj_record[q][0] and chroms_obj_record[p][1]<chroms_obj_record[q][1])
            or (chroms_obj_record[p][0]>chroms_obj_record[q][0] and chroms_obj_record[p][1]<=chroms_obj_record[q][1])):
                if q not in s[p]:
                    s[p].append(q)
            
            # if q dominate p
            elif ((chroms_obj_record[p][0]<chroms_obj_record[q][0] and chroms_obj_record[p][1]>chroms_obj_record[q][1]) 
            or (chroms_obj_record[p][0]<=chroms_obj_record[q][0] and chroms_obj_record[p][1]>chroms_obj_record[q][1])
            or (chroms_obj_record[p][0]<chroms_obj_record[q][0] and chroms_obj_record[p][1]>=chroms_obj_record[q][1])):
                n[p]=n[p]+1
        
        if n[p]==0:
            rank[p]=0
            if p not in front[0]:
                front[0].append(p)
    

    # Create Front by rank
    i=0
    while (front[i]!=[]):
        Q=[]
        for p in front[i]:
            for q in s[p]:
                n[q]=n[q]-1
                if n[q]==0:
                    rank[q]=i+1
                    if q not in Q:
                        Q.append(q)
        i=i+1
        front[i]=Q
                
    del front[len(front)-1]
    return front


In [ ]:
###################################################
#-----Calculate crowding distance function---------
###################################################

def calculate_crowding_distance(front,chroms_obj_record):
    
    distance={m:0 for m in front}
    for o in range(2):
        obj={m:chroms_obj_record[m][o] for m in front}
        sorted_keys=sorted(obj, key=obj.get)
        distance[sorted_keys[0]]=distance[sorted_keys[len(front)-1]]=999999999999 # Why latest front = 999999
        for i in range(1,len(front)-1):
            if len(set(obj.values()))==1:
                distance[sorted_keys[i]]=distance[sorted_keys[i]]
            else:
                distance[sorted_keys[i]]=distance[sorted_keys[i]]+(obj[sorted_keys[i+1]]-obj[sorted_keys[i-1]])/(obj[sorted_keys[len(front)-1]]-obj[sorted_keys[0]])
            
    return distance  

In [7]:
obj = {3:6,1: 1,2:1}
sorted(obj, key=obj.get)

[1, 2, 3]

In [18]:
# Ngẫu nhiên assign resouce to Task
population_list=[]
population_size = 10
num_task = 10
num_vm = 3
for i in range(population_size):
    #print('i=',i)
    nxm_random_num=list(np.random.permutation(num_task)) # generate a random permutation of 0 to num_job*num_mc-1
    population_list.append(nxm_random_num) # add to the population_list
    for j in range(num_task):
        population_list[i][j]=population_list[i][j]%num_vm # convert to job number format, every job appears m times

population_list

[[0, 1, 2, 1, 1, 0, 2, 0, 2, 0],
 [0, 2, 1, 1, 0, 2, 2, 0, 1, 0],
 [1, 2, 0, 0, 1, 0, 2, 2, 0, 1],
 [0, 1, 2, 2, 1, 1, 0, 2, 0, 0],
 [0, 0, 1, 1, 0, 2, 2, 0, 1, 2],
 [0, 1, 1, 1, 0, 0, 2, 0, 2, 2],
 [0, 0, 1, 0, 2, 1, 2, 2, 0, 1],
 [1, 0, 2, 2, 0, 1, 0, 2, 1, 0],
 [1, 0, 0, 2, 1, 1, 2, 0, 0, 2],
 [0, 0, 1, 1, 0, 2, 1, 0, 2, 2]]

In [38]:
list(np.random.permutation(num_task))

[9, 2, 6, 5, 7, 4, 1, 8, 3, 0]

In [10]:
###################################################
#-----Main-----------------------------------------
###################################################
start_time = time.time() 

###################################################
#-----Generate initial population------------------
###################################################
best_list,best_obj=[],[]


        
for n in range(num_iteration):         
    ###################################################
    #-----Crossover assign VM to task------------------------------------
    ###################################################
    parent_list=copy.deepcopy(population_list)
    offspring_list=[] # danh sách children
    
    S=list(np.random.permutation(population_size)) # generate a random sequence to select the parent chromosome to crossover
    
    for m in range(int(population_size/2)):
        
        parent_1= population_list[S[2*m]][:]
        parent_2= population_list[S[2*m+1]][:]
        child_1=parent_1[:]
        child_2=parent_2[:]
        
        cutpoint=list(np.random.choice(num_task, 2, replace=False))
        cutpoint.sort()
    
        child_1[cutpoint[0]:cutpoint[1]]=parent_2[cutpoint[0]:cutpoint[1]]
        child_2[cutpoint[0]:cutpoint[1]]=parent_1[cutpoint[0]:cutpoint[1]]
        
        offspring_list.extend((child_1,child_2))

    ###################################################
    #-----Mutation-------------------------------------
    ###################################################
    for m in range(len(offspring_list)):
        mutation_prob=np.random.rand()
        if mutation_rate <= mutation_prob:
            m_chg=list(np.random.choice(num_task, num_mutation_jobs, replace=False)) # chooses the position to mutation
            t_value_last=offspring_list[m][m_chg[0]] # save the value which is on the first mutation position
            for i in range(num_mutation_jobs-1):
                offspring_list[m][m_chg[i]]=offspring_list[m][m_chg[i+1]] # displacement
            
            offspring_list[m][m_chg[num_mutation_jobs-1]]=t_value_last
                
    ###################################################
    #-----Fitness valuse ------------------------------
    ###################################################               
    total_chromosome=copy.deepcopy(parent_list)+copy.deepcopy(offspring_list)
    chroms_obj_record={} 
    for m in range(population_size*2):
        gen_c=0
        gen_r=1
        for nn in range(num_task):
            # print(total_chromosome[m][nn])
            gen_c +=Task_Vm_Cost[nn][total_chromosome[m][nn]]
            gen_r *=Task_Vm_Reliability[nn][total_chromosome[m][nn]]
        chroms_obj_record[m]=[gen_r,gen_c]
        
    ###################################################
    #-----Non-dominated sorting -----------------------
    ################################################### 
    front=non_dominated_sorting(population_size,chroms_obj_record)
    
    ###################################################
    #-----Selection -----------------------------------
    ###################################################         
    population_list,new_pop=selection(population_size,front,chroms_obj_record,total_chromosome)
    new_pop_obj=[chroms_obj_record[k] for k in new_pop] 
    
    ###################################################
    #-----Comparison ----------------------------------
    ################################################### 
    if n==0:
        best_list=copy.deepcopy(population_list)
        best_obj=copy.deepcopy(new_pop_obj)
    else:            
        total_list=copy.deepcopy(population_list)+copy.deepcopy(best_list)
        total_obj=copy.deepcopy(new_pop_obj)+copy.deepcopy(best_obj)
        
        now_best_front=non_dominated_sorting(population_size,total_obj)
        best_list,best_pop=selection(population_size,now_best_front,total_obj,total_list)
        best_obj=[total_obj[k] for k in best_pop]
###################################################
#-----Results ------------------------------------
###################################################
print('-----Results -----------------------------')
print("One chromosome(1x100)=",best_list[0])
print("[Reliability,Cost]=",best_obj[0])
print("------------------------------------------")
print('The elapsed time:%s'% (time.time() - start_time))

-----Results -----------------------------
One chromosome(1x100)= [8, 8, 3, 7, 1, 3, 9, 9, 3, 3, 4, 8, 0, 6, 6, 1, 7, 4, 0, 2, 1, 1, 9, 8, 2, 9, 2, 3, 2, 0]
[Reliability,Cost]= [0.8021592554949589, 9.54531862421403]
------------------------------------------
The elapsed time:29.8622989654541


In [28]:
list(np.random.choice(10, 2, replace=False))

[4, 1]

## Apply NSGA2

In [52]:
# Initial Resouces
NUM_WORKFORCE = 10
NUM_TASK = 30

# Initial NSGA2 params
POPULATION_SIZE = 100
NUMBER_ITERATION = 500
CROSSOVER_RATE = 0.8

MUTATION_SELECTION_RATE = 0.4


In [51]:
round(NUM_TASK*MUTATION_SELECTION_RATE)

12

### Initial workforce to task

In [44]:
# Random assign workforce to task
population_list=[]
for i in range(POPULATION_SIZE):
    nxm_random_num=list(np.random.permutation(NUM_TASK)) # generate a random permutation of 0 to num_job*num_mc-1
    population_list.append(nxm_random_num) # add to the population_list
    for j in range(num_task):
        population_list[i][j]=population_list[i][j]%NUM_WORKFORCE # convert to job number format, every job appears m times

### NSGA2 Method

In [63]:
def crossover(population_list):
    cosssover_offspring = [] # danh sách con cái được tạo ra từ crossover
    # Lấy ngẫu nhiên 2 parents
    random_parent_indices =list(np.random.permutation(POPULATION_SIZE)) # generate a random sequence to select the parent chromosome to crossover
    
    for m in range(int(POPULATION_SIZE*CROSSOVER_RATE)):
        parent_1= population_list[random_parent_indices[2*m]][:]
        parent_2= population_list[random_parent_indices[2*m+1]][:]
        child_1=parent_1[:]
        child_2=parent_2[:]
        
        # Lấy ngẫu nhiên 2 đoạn bất kỳ trong parent 1 chuyển cho parent 2
        cutpoint=list(np.random.choice(num_task, 2, replace=False))
        cutpoint.sort()
        child_1[cutpoint[0]:cutpoint[1]]=parent_2[cutpoint[0]:cutpoint[1]]
        child_2[cutpoint[0]:cutpoint[1]]=parent_1[cutpoint[0]:cutpoint[1]]
        
        cosssover_offspring.extend((child_1,child_2))
    return cosssover_offspring


def mutation(offspring_list):
    num_mutation_jobs = round(NUM_TASK*MUTATION_SELECTION_RATE)
    
    # Thực hiện đảo ngược 1 gen
    for m in range(len(offspring_list)):
        mutation_prob=np.random.rand()
        if mutation_rate <= mutation_prob:
            # chooses the position to mutation
            m_chg=list(np.random.choice(num_task, num_mutation_jobs, replace=False))
            # save the value which is on the first mutation position
            t_value_last=offspring_list[m][m_chg[0]] 
            
            for i in range(num_mutation_jobs-1):
                offspring_list[m][m_chg[i]]=offspring_list[m][m_chg[i+1]] # displacement
            
            offspring_list[m][m_chg[num_mutation_jobs-1]]=t_value_last

    return offspring_list


def selection(front,chroms_obj_record,total_chromosome):   
    N=0
    new_pop=[]
    while N < POPULATION_SIZE:
        for i in range(len(front)):
            N=N+len(front[i])
            if N > POPULATION_SIZE:
                # Compare on each front after append solution with higher distance to new_pop
                distance=calculate_crowding_distance(front[i],chroms_obj_record)
                sorted_cdf=sorted(distance, key=distance.get)
                sorted_cdf.reverse()
                for j in sorted_cdf:
                    if len(new_pop)==population_size:
                        break                
                    new_pop.append(j)              
                break
            else:
                new_pop.extend(front[i])
    
    population_list=[]
    for n in new_pop:
        population_list.append(total_chromosome[n])
    
    return population_list,new_pop

    

### Objective

In [ ]:
TASK_WORKFORCE_COST = None
TASK_EQUIPMENT_COST = None
TASK_SKILL_LEVEL = None
TASK_SIMILARITY_MATRIX = None

def cost_obj(chromosome):
    workforce_cost = 0
    equipment_cost = 0
    for n in range(chromosome):
        workforce_cost += TASK_WORKFORCE_COST[n][chromosome[n]]
        equipment_cost += TASK_EQUIPMENT_COST[n][chromosome[n]]
    
    return equipment_cost + workforce_cost


def assigned_level_obj(chromosome):
    skill_level_total = 0
    for n in range(chromosome):
        skill_level_total += TASK_SKILL_LEVEL[n][chromosome[n]]
    return skill_level_total


def similarity_task_obj(chromosome):
    # TASK_SIMILARITY_MATRIX[]
    pass


def complete_time_obj(chromosome):
    # compute_complete_time
    pass

### Fitness

In [62]:
def compute_fitness(total_chromosome):
    chroms_obj_record={} 
    for m in range(POPULATION_SIZE*2):
        cost = cost_obj(total_chromosome[m])
        assigned_level = assigned_level_obj(total_chromosome[m])
        similarity_task = similarity_task_obj(total_chromosome[m])
        complete_time = complete_time_obj(total_chromosome[m])

        chroms_obj_record[m]=[cost,1/assigned_level, 1/similarity_task, 1/complete_time]

    return chroms_obj_record

### Run

In [ ]:
start_time = time.time() 

best_list,best_obj=[],[]
for n in range(num_iteration):         

    parent_list=copy.deepcopy(population_list)
    offspring_list=[] # danh sách children
    
    # Crossover
    crossover_offspring = crossover(population_list = population_list)
    offspring_list.extend(crossover_offspring)

    # Mutation
    offspring_list = mutation(offspring_list = offspring_list)
    
    # Selection
    total_chromosome=copy.deepcopy(parent_list)+copy.deepcopy(offspring_list)
    chroms_obj_record = compute_fitness(total_chromosome)
    front=non_dominated_sorting(population_size,chroms_obj_record)
 
    population_list,new_pop=selection(population_size,front,chroms_obj_record,total_chromosome)
    new_pop_obj=[chroms_obj_record[k] for k in new_pop] 
    
    # Comparison
    if n==0:
        best_list=copy.deepcopy(population_list)
        best_obj=copy.deepcopy(new_pop_obj)
    else:            
        total_list=copy.deepcopy(population_list)+copy.deepcopy(best_list)
        total_obj=copy.deepcopy(new_pop_obj)+copy.deepcopy(best_obj)
        
        now_best_front=non_dominated_sorting(population_size,total_obj)
        best_list,best_pop=selection(population_size,now_best_front,total_obj,total_list)
        best_obj=[total_obj[k] for k in best_pop]


print('-----Results -----------------------------')
print("One chromosome(1x100)=",best_list[0])
print("[Reliability,Cost]=",best_obj[0])
print("------------------------------------------")
print('The elapsed time:%s'% (time.time() - start_time))